In [ ]:
# 必要なライブラリのインストール
!pip install cloudmlmagic keras
%load_ext cloudmlmagic 

In [3]:
# Cloud Magickの初期化
%%ml_init -projectId asobiba-196608 -bucket asobiba-ml-bucket -scaleTier BASIC_GPU -region asia-east1 -runtimeVersion 1.2
{'install_requires': ['keras', 'h5py', 'Pillow']}

In [ ]:
%%ml_code

# モデル定義

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from io import BytesIO
import numpy as np
import pandas as pd
import requests

model = InceptionV3(weights='imagenet')

url = 'https://storage.googleapis.com/asobiba-ml-bucket/datasets.npz'
response = requests.get(url)
dataset = np.load(BytesIO(response.content))

X_dataset = dataset['features']
y_dataset = dataset['labels']

X_dataset = preprocess_input(X_dataset)
y_dataset = np_utils.to_categorical(y_dataset)
X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, y_dataset, test_size=0.2, random_state=42)

# 転移学習用のモデル
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[311].output)

# レイヤー追加
x = intermediate_layer_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Transfer learning model, all layers are trainable at this moment
transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

print(pd.DataFrame(transfer_model.layers).tail())

# Freeze all layers
for layer in transfer_model.layers:
    layer.trainable = False

# Unfreeze the last layers, so that only these layers are trainable.
transfer_model.layers[312].trainable = True
transfer_model.layers[313].trainable = True

transfer_model.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])

In [9]:
%%ml_run cloud

# モデル学習

import tensorflow as tf
from keras import backend as K

transfer_model.fit(X_train, y_train, epochs=30,
                   verbose=2,
                   validation_data=(X_test, y_test))
loss, acc = transfer_model.evaluate(X_test, y_test)
print('Loss {}, Accuracy {}'.format(loss, acc))

K.set_learning_phase(0)  # test
sess = K.get_session()

from tensorflow.python.framework import graph_util

# Make GraphDef of Transfer Model
g_trans = sess.graph
g_trans_def = graph_util.convert_variables_to_constants(sess,
                                                        g_trans.as_graph_def(),
                                                        [transfer_model.output.name.replace(':0','')])

# Image Converter Model
with tf.Graph().as_default() as g_input:
    input_b64 = tf.placeholder(shape=(1,), dtype=tf.string, name='input')
    input_bytes = tf.decode_base64(input_b64[0])
    image = tf.image.decode_image(input_bytes)
    image_f = tf.image.convert_image_dtype(image, dtype=tf.float32)
    input_image = tf.expand_dims(image_f, 0)
    output = tf.identity(input_image, name='input_image')

g_input_def = g_input.as_graph_def()

with tf.Graph().as_default() as g_combined:
    x = tf.placeholder(tf.string, name="input_b64")

    im, = tf.import_graph_def(g_input_def,
                              input_map={'input:0': x},
                              return_elements=["input_image:0"])

    pred, = tf.import_graph_def(g_trans_def,
                                input_map={transfer_model.input.name: im,
                                          'batch_normalization_1/keras_learning_phase:0': False},
                                return_elements=[transfer_model.output.name])

    pred_index = tf.argmax(pred, axis=-1)
    books = tf.constant([ "serverless", "production_ready", "devops" ], name="label")
    book_name = tf.gather(books, pred_index)

    with tf.Session() as sess2:
        inputs = {"inputs": tf.saved_model.utils.build_tensor_info(x)}
        outputs = {"pred_index": tf.saved_model.utils.build_tensor_info(pred_index), 
                   "score": tf.saved_model.utils.build_tensor_info(pred), 
                   "book_name": tf.saved_model.utils.build_tensor_info(book_name)}
        signature = tf.saved_model.signature_def_utils.build_signature_def(
            inputs=inputs,
            outputs=outputs,
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
        )

        # save as SavedModel
        b = tf.saved_model.builder.SavedModelBuilder('gs://asobiba-ml-bucket/keras-mlengine/savedmodel')
        b.add_meta_graph_and_variables(sess2,
                                       [tf.saved_model.tag_constants.SERVING],
                                       signature_def_map={'serving_default': signature})
        b.save()

{u'trainingOutput': {}, u'state': u'QUEUED', u'trainingInput': {u'runtimeVersion': u'1.2', u'packageUris': [u'gs://asobiba-ml-bucket/mlmagic__1524471999.tar.gz'], u'region': u'asia-east1', u'pythonModule': u'trainer.task', u'scaleTier': u'BASIC_GPU'}, u'createTime': u'2018-04-23T08:27:01Z', u'jobId': u'mlmagic__1524471999'}


In [10]:
# 一時停止用
raise "stop"

TypeError: exceptions must be old-style classes or derived from BaseException, not str

In [ ]:
# Cloud ML Enginge のモデルおよびAPI作成
!gcloud ml-engine models create MLBooks
!gcloud ml-engine versions create v5 --model MLBooks --runtime-version 1.2 --origin gs://asobiba-ml-bucket/keras-mlengine/savedmodel

In [ ]:
# 予測テスト
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

PROJECTID = 'asobiba-196608'
projectID = 'projects/{}'.format(PROJECTID)
modelName = 'MLBooks'
modelID = '{}/models/{}'.format(projectID, modelName)

credentials = GoogleCredentials.get_application_default()
ml = discovery.build('ml', 'v1', credentials=credentials)

with open('input/s_serverless_image.jpg', 'rb') as f:
    b64_x = f.read()

import base64
import json

b64_x = base64.urlsafe_b64encode(b64_x)
input_instance = dict(inputs=b64_x)
input_instance = json.loads(json.dumps(input_instance))
request_body = {"instances": [input_instance]}

request = ml.projects().predict(name=modelID, body=request_body)
try:
    response = request.execute()
except errors.HttpError as err:
    # Something went wrong with the HTTP transaction.
    # To use logging, you need to 'import logging'.
    print('There was an HTTP error during the request:')
    print(err._get_reason())
response